In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import the numpy, pandas and tensorflow libraries to work with arrays, matrices, and neural networks, respectively

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
tf.__version__

'2.14.0'

Import the dataset and create two vectors

In [4]:
dataset = pd.read_csv('/content/drive/My Drive/ANN/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [5]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


Import LabelEncoder from the sklearn.preprocessing library, then pass the values of the Gender column in its fit_transform function

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


Import OneHotEncoder from the sklearn.preprocessing library to use on the Geography column

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder= 'passthrough')
X = np. array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Split into training and testing sets for independent and dependent variables

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

Import StandardScaler to be able to use the fit_transform() and transform() functions. Use feature scaling when building neural network related models

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc. transform(X_test)

Use Sequential class to initialize the artificial neural network

In [12]:
ann = tf.keras.models.Sequential()

Add the input layer and the first hidden layer using the add() method. Inside the add method use the Dense class and specify the number of neurons and type of activation function as parameters.

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Add the second hidden layer using the same function and class

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Add the output layer and specify the number of neurons as 1 since it is binary classification, and the activation function as sigmoid.

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Compile the ANN using the compile() method, and specify the optimizer, loss function and metrics parameters. The loss function calculates the difference between the predicted and actual values.

In [16]:
ann.compile(optimizer ="adam" , loss = 'binary_crossentropy', metrics = ['accuracy'])

Use the fit method to train the ANN on the training set. Specify the batch size to be 32 to compare data in batches. Also specify the epochs to be 100 to ensure the model learns through the 100 iterations.

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
235/235 [==============================] - 2s 3ms/step - loss: 0.5922 - accuracy: 0.7305
Epoch 2/100
235/235 [==============================] - 1s 4ms/step - loss: 0.4808 - accuracy: 0.7965
Epoch 3/100
235/235 [==============================] - 1s 4ms/step - loss: 0.4481 - accuracy: 0.7969
Epoch 4/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4359 - accuracy: 0.8011
Epoch 5/100
235/235 [==============================] - 1s 4ms/step - loss: 0.4290 - accuracy: 0.8069
Epoch 6/100
235/235 [==============================] - 1s 5ms/step - loss: 0.4228 - accuracy: 0.8143
Epoch 7/100
235/235 [==============================] - 1s 5ms/step - loss: 0.4167 - accuracy: 0.8181
Epoch 8/100
235/235 [==============================] - 1s 5ms/step - loss: 0.4099 - accuracy: 0.8239
Epoch 9/100
235/235 [==============================] - 1s 5ms/step - loss: 0.4026 - accuracy: 0.8284
Epoch 10/100
235/235 [==============================] - 1s 6ms/step - loss: 0.3938 - accura

Use the model to predict if a customer with the following information will leave the bank. In this case the prediction is false, which means the customer will not leave the bank. Since the values were scaled when training the ANN, the same has to be done here

In [18]:
print (ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 83ms/step
[[False]]


To predict the test results, define the threshold value again, and use the concatenate() method to join together the y_pred and y_test columns.

In [19]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

79/79 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Generate the confusion matrix to display true and false positives/negatives. Then calculate the accuracy_score using y_test and y_pred.

In [20]:
from sklearn.metrics import confusion_matrix, accuracy_score
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)
accuracy_score(y_test, y_pred)

[[1865  126]
 [ 236  273]]


0.8552